In [1]:
import numpy as np
from itertools import product

# 3. Mean Field Approximation and Gibbs Sampling

## Part 1: Exact Inference

Given a binary variable Markov Random Field with parameter $\beta$: $p(x) = Z^{-1}\prod_{i>j}\phi(x_i,x_j)$, defined on the n x n lattice with $\phi(x_i,x_j) = e^{\beta I[x_i=x_j]}$ for i a neighbour of j on the lattice and i > j.

You will need to compute the joint probability distribution of the top and bottom nodes of the rightmost column of the 10 × 10 lattice for $\beta = 4$, $\beta = 1$, and $\beta = 0.01.$ In other words, need to find probability table for $P(x_{1,10}, x_{10,10})$

Perform exact inference, using techniques from Exercise 6.7. That is, treat
each column as one variable with 2n states and perform message passing
on the induced factor-graph. [5 marks]

---- 
The joint for this MRF will be $P(V_1, V_2,.. V_{10}) \propto \Phi_1(V_1) * \Psi_1(V_1, V_2)* \Phi_2(V_2) *...* \Psi_{9}(V_9, V_{10})* \Phi_{10}(V_{10})$

To find this, we need to find $\Phi_t(V_t)$ (the internal potentials, describe verticals) and $\Psi_t(V_t, V_{t+1})$ (the transition potentials, describe horizontal, pairwise). $\Phi_t(V_t)$ will be a vector with $2^{10}=1024$ states, and $\Psi_t(V_t,V_{t+1})$ will be a matrix with $2^{10}$ x $2^{10}=1024$ x $1024$ states describing the probability of transition from one of the 1024 states in $V_t$ to one of the 1024 states in $V_{t+1}$.

We can define $\Phi_t(V_t) = \prod_{i=1}^{9}\phi(x_{i,t},x_{i+1,t})$, and $\Psi_t(V_t, V_{t+1}) = \prod_{i=1}^{10}\phi(x_{i,t},x_{i,t+1})$.

Then, to calculate $P(x_{1,10}, x_{10,10})$, we will need to use the Sum-Product algorithm, where  


In [2]:
def phi(beta, xi, xj):
    """
    Function to calculate phi(xi,xj) for a given beta
    """
    indicator = np.sum(xi == xj)
    return np.exp(beta * indicator)


def calculate_Phi_Psi(N, K, states, beta):
    """
    Function to calculate the potentials Phi(Vt) and Psi(Vt, Vt+1) for an 
    N x N lattice Markov Random Field. 
    """
    # create 2^N x N matrix of all potential values of binary vector Vt (one column of lattice)
    all_combinations_tuple = product(states, repeat=N)
    V_t_matrix = np.array(list(all_combinations_tuple))

    # initialize Phi_t(Vt), this will be a 2^10 x 2^10 array
    Phi_t = np.zeros(K)

    # for each potential column state k of Vt, find Phi(Vt)
    for k in range(K):
        # this will be a single state of vector Vt of shape (N, )
        V_t_state = V_t_matrix[k, :]
        # calculate the value of Phi(Vt) for given state
        Phi_t[k] = phi(beta,V_t_state[:-1],V_t_state[1:])

    # initialize Psi_t(Vt, Vt+1), this will be a 2^10 x 2^10 matrix 
    Psi_t = np.zeros((K, K))

    # for each potential column k_t of state Vt and each potential
    # column k_tplus1 of Vt+1, find Psi(Vt, Vt+1)
    for k_t in range(K):
        # same single state of vector Vt of shape (N, ) as above
        V_t_state = V_t_matrix[k_t, :]
        for k_tplus1 in range(K):
            # single state of vector Vt+1 of shape (N, )
            V_tplus1_state = V_t_matrix[k_tplus1, :]
            Psi_t[k_t, k_tplus1] = phi(beta, V_t_state, V_tplus1_state)

    print(f'Phi_t(Vt) Shape: {Phi_t.shape}')
    print(f'Psi_t(Vt, Vt+1) Shape: {Psi_t.shape}\n')

    return (Phi_t, Psi_t)

In [3]:
def belief_propogation(Phi, Psi, T, i, j, V_t_matrix, N, K, states):
    """
    Belief propogation to calculate joint probability of P(Xi,T, Xj,T).
    """
    # initialize alphas (forward pass), a vector of length K (column states)
    alphas = np.zeros((T, K))
    alphas[0] = Phi
    alphas[0] /= np.sum(alphas[0])

    # for each timestep from 1 to T...
    for t in range(1, T):
        # message from t-1 and potentials for Vt
        alpha_t = Phi * (Psi.T @ alphas[t-1]) # matrix of shape (K x K) @ vector of shape (K, ) = vector of shape (K, )
        # find normalization constant for t
        Z_t = np.sum(alpha_t)
        alphas[t] = alpha_t/Z_t

    # backward pass
    betas = np.zeros((T, K))
    betas[T-1] = np.ones(K)

    for t in reversed(range(T-1)):
        # message from t+1 to t for Vts
        beta_t = Psi @ (betas[t+1] * Phi )
        # find normalization constant for t
        Z_t = np.sum(beta_t)
        betas[t] = beta_t/Z_t

    # calculate belief vectors for Vt at final timestep t=T, shape (K, )
    belief_T = (alphas[T-1] * betas[T-1])/np.sum((alphas[T-1] * betas[T-1]))

    # to get joint prob, need to find relevant belief vectors (where states line up with
    # desired Xi,j
    joint_distribution = np.zeros((len(states),len(states)))
    for idx1, s1 in enumerate(states):
        for idx2, s2 in enumerate(states):
            # find indices where 
            indices = np.logical_and(V_t_matrix[:,i-1] == s1, V_t_matrix[:,j-1] == s2)

            unnormalized_joint = np.sum(belief_T[indices])

            joint_distribution[idx1, idx2] = unnormalized_joint
    return joint_distribution

In [4]:
# define dimensions of square lattice
N = 10
betas = [4, 1, 0.01]

# define states of xi,j to be [0,1]
states = [0,1]
# column states (K)
K = len(states)**N

print(f'There are {K} x {K} = {K**2} potential states for the {N} x {N} lattice of the binary Markov Random Field.\n')

print(f'To reduce computational complexity, we will consider {N} vectors of length {N}, each with {K} potential states.\n')

# create 2^N x N matrix of all potential values of binary vector Vt (one column of lattice)
all_combinations_tuple = product(states, repeat=N)
V_t_matrix = np.array(list(all_combinations_tuple))

# loop through each value of beta
for beta in betas:
    Phi_t, Psi_t = calculate_Phi_Psi(N=N, K=K, states=states, beta=beta)

    p_joint = belief_propogation(Phi_t, Psi_t, T=3, i=1, j=10, V_t_matrix=V_t_matrix, N=N, K=K, states=states)
    print(f'Joint Probability Distribution of P(X1,10, X10,10) for Beta = {beta}:')
    print(p_joint)

There are 1024 x 1024 = 1048576 potential states for the 10 x 10 lattice of the binary Markov Random Field.

To reduce computational complexity, we will consider 10 vectors of length 10, each with 1024 potential states.

Phi_t(Vt) Shape: (1024,)
Psi_t(Vt, Vt+1) Shape: (1024, 1024)

Joint Probability Distribution of P(X1,10, X10,10) for Beta = 4:
[[4.99622685e-01 3.77315354e-04]
 [3.77315354e-04 4.99622685e-01]]
Phi_t(Vt) Shape: (1024,)
Psi_t(Vt, Vt+1) Shape: (1024, 1024)

Joint Probability Distribution of P(X1,10, X10,10) for Beta = 1:
[[0.25974154 0.24025846]
 [0.24025846 0.25974154]]
Phi_t(Vt) Shape: (1024,)
Psi_t(Vt, Vt+1) Shape: (1024, 1024)

Joint Probability Distribution of P(X1,10, X10,10) for Beta = 0.01:
[[0.25 0.25]
 [0.25 0.25]]


$\beta$ = 4

| $\beta = 4$  | $P(x_{10,10}) = 0$| $P(x_{10,10}) =1$|
| -------- | ------- | ------- |
| $P(x_{1,10}) =0$        | 0.4997 | 0.0004 |
| $P(x_{1,10}) =1$        | 0.0004 | 0.4997 |

$\beta$ = 1

| $\beta = 4$  | $P(x_{10,10}) = 0$| $P(x_{10,10}) =1$|
| -------- | ------- | ------- |
| $P(x_{1,10}) =0$        | 0.2808 | 0.2192 |
| $P(x_{1,10}) =1$        | 0.2192 | 0.2808 |

$\beta$ = 0.01

| $\beta = 4$  | $P(x_{10,10}) = 0$| $P(x_{10,10}) =1$|
| -------- | ------- | ------- |
| $P(x_{1,10}) =0$        | 0.25 | 0.25 |
| $P(x_{1,10}) =1$        | 0.25 | 0.25 |